<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

Import scopes are divided in several notebooks:
- [Calculation of Identities](../Import/1ImportScope-Identities)
- [Calculation of Present Values](../Import/2ImportScope-PresentValue)
- [Calculation of Actuals](../Import/3ImportScope-Actuals)
- [Calculation of Technical Margin](../Import/4ImportScope-TechnicalMargin)
- [Creation of Ifrs Variables](../Import/5ImportScope-ToIfrsVar)
- Calculation of Ifrs Variables

<br><br>
In this notebook the focus is on the calculation of the minimal set of Ifrs Variable given the import Format.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "5ImportScope-ToIfrsVar"

# Actual dependent variables

In [0]:
public interface ComputeIfrsVarsActuals : ActualToIfrsVariable, DeferrableToIfrsVariable, EaForPremiumToIfrsVariable, TmToIfrsVariable
{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => Actual.Concat(AdvanceActual).Concat(OverdueActual)
        .Concat(ActEAForPremium).Concat(Deferrable).Concat(Csms).Concat(Loss);
}

# Cashflow dependent variables

In [0]:
public interface ComputeIfrsVarsCashflows : PvToIfrsVariable, RaToIfrsVariable, DeferrableToIfrsVariable, EaForPremiumToIfrsVariable, 
    TmToIfrsVariable, NominalToIfrsVariable, RevenueToIfrsVariable, ActualToIfrsVariable
{
    IEnumerable<IfrsVariable> amortizationFactors => Identity.ValuationApproach switch {
        ValuationApproaches.PAA => AmortizationFactor.Union(DeferrableAmFactor, EqualityComparer<IfrsVariable>.Instance),
//.Union(RevenueAmFactor, EqualityComparer<IfrsVariable>.Instance),//No need to RevenueAmFactor as long as Revenue depends only on Cashflow
        _ => AmortizationFactor.Union(DeferrableAmFactor, EqualityComparer<IfrsVariable>.Instance),
    };
    
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => ( (Identity.ValuationApproach, GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType) switch {
        (ValuationApproaches.VFA, _) => PvCurrent.Concat(CumulatedNominal).Concat(RaCurrent).Concat(amortizationFactors)
            .Concat(BeEAForPremium).Concat(Deferrable).Concat(Csms).Concat(Loss),
        (ValuationApproaches.PAA, LiabilityTypes.LIC) => PvLocked.Concat(PvCurrent).Concat(CumulatedNominal).Concat(RaCurrent).Concat(RaLocked),                           
        (ValuationApproaches.PAA, LiabilityTypes.LRC) => PvLocked.Concat(CumulatedNominal).Concat(RaLocked).Concat(amortizationFactors).Concat(BeEAForPremium)
            .Concat(Deferrable).Concat(Loss).Concat(Revenue),
        _ => PvLocked.Concat(PvCurrent).Concat(CumulatedNominal).Concat(RaCurrent).Concat(RaLocked).Concat(amortizationFactors)
             .Concat(BeEAForPremium).Concat(Deferrable).Concat(Csms).Concat(Loss),
    } )
    // Adding Actuals.
    .Concat(Actual).Concat(ActEAForPremium);
}

# Opening dependent variables

In [0]:
public interface ComputeIfrsVarsOpenings : ActualToIfrsVariable, DeferrableToIfrsVariable, TmToIfrsVariable
{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => AdvanceActual.Concat(OverdueActual)
        .Concat(Deferrable).Concat(Csms).Concat(Loss);//.Concat(Revenue)?
}

# Inject Projections

In [0]:
public interface ProjectionCount : IScope<string, ImportStorage>{
    private int rawVariableMaxLength => GetStorage().GetRawVariables(Identity).Max(rawVariable => rawVariable.Values.Length);
    int Value => Enumerable.Range(0, GetStorage().GetProjectionCount()).Where(projCount => 
        rawVariableMaxLength > GetStorage().GetShift(projCount)).Count();
}

In [0]:
public interface ComputeIfrsVarsCashflowsWithProjection : IScope<string, ImportStorage>{
    private IEnumerable<ImportIdentity> identities => GetScope<GetIdentities>(Identity).Identities;
    
    private IEnumerable<ImportIdentity> identitiesWithProjection => Enumerable.Range(0, GetScope<ProjectionCount>(Identity).Value)
        .SelectMany(projectionPeriod => identities.Select(id => id with { ProjectionPeriod = projectionPeriod})); //temp?
    
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => identitiesWithProjection
        .SelectMany(identity => GetScope<ComputeIfrsVarsCashflows>(identity).CalculatedIfrsVariables)
        .AggregateProjections()
        .Select(ifrsVariable => ifrsVariable with {Partition = GetStorage().TargetPartition});
}

In [0]:
public interface ComputeIfrsVarsActualsWithProjection : IScope<string, ImportStorage>{
    private IEnumerable<ImportIdentity> identities => GetScope<GetIdentities>(Identity).Identities;
    
    private IEnumerable<ImportIdentity> identitiesWithProjection => Enumerable.Range(0, GetScope<ProjectionCount>(Identity).Value)
        .SelectMany(projectionPeriod => identities.Select(id => id with { ProjectionPeriod = projectionPeriod})); //temp?
    
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => identitiesWithProjection
        .SelectMany(identity => GetScope<ComputeIfrsVarsActuals>(identity).CalculatedIfrsVariables)
        .AggregateProjections()
        .Select(ifrsVariable => ifrsVariable with {Partition = GetStorage().TargetPartition});
}

In [0]:
public interface ComputeIfrsVarsOpeningsWithProjection : IScope<string, ImportStorage>{
    private IEnumerable<ImportIdentity> identities => GetScope<GetIdentities>(Identity).Identities;
    
    private IEnumerable<ImportIdentity> identitiesWithProjection => Enumerable.Range(0, GetScope<ProjectionCount>(Identity).Value)
        .SelectMany(projectionPeriod => identities.Select(id => id with { ProjectionPeriod = projectionPeriod})); //temp?
    
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => identitiesWithProjection
        .SelectMany(identity => GetScope<ComputeIfrsVarsOpenings>(identity).CalculatedIfrsVariables)
        .AggregateProjections()
        .Select(ifrsVariable => ifrsVariable with {Partition = GetStorage().TargetPartition});
}

# Compute 

In [0]:
public interface ComputeAllScopes: IScope<string, ImportStorage>
{
   IEnumerable<IfrsVariable> CalculatedIfrsVariables => GetStorage().ImportFormat switch {
            ImportFormats.Actual   => GetScope<ComputeIfrsVarsActualsWithProjection >(Identity).CalculatedIfrsVariables,
            ImportFormats.Cashflow => GetScope<ComputeIfrsVarsCashflowsWithProjection>(Identity).CalculatedIfrsVariables,
            ImportFormats.Opening  => GetScope<ComputeIfrsVarsOpeningsWithProjection>(Identity).CalculatedIfrsVariables,
   };
}